In [7]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


#Reading Data from CSV Files
temperature_readings = pd.read_csv('data/W512_readings_floorplan1.csv') #original dataset for temp readings 
temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


weather_readings = pd.read_csv('data/weatherData.csv') #orignal dataset for weather readings

# Converting date and time in temperature_readings to UNIX timestamp for comparision 
temperature_readings['datetime_str'] = temperature_readings['date'] + ' ' + temperature_readings['time']
temperature_readings['datetime'] = temperature_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
temperature_readings['unix_timestamp'] = temperature_readings['datetime'].apply(lambda x: int(x.timestamp()))

# Converting date and time in weather_readings to UNIX timestamp for comparision 
weather_readings['datetime_str'] = weather_readings['date'] + ' ' + weather_readings['time']
weather_readings['datetime'] = weather_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
weather_readings['unix_timestamp'] = weather_readings['datetime'].apply(lambda x: int(x.timestamp()))

weather_result_col = [col for col in weather_readings.columns if "result" in col.lower()]

# Merging both data for it to be on the same time
merged_data = pd.merge_asof(
    temperature_readings,  # Left DataFrame
    weather_readings[['unix_timestamp']+weather_result_col],      # Right DataFrame
    on='unix_timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

datetime_string = merged_data['date'] + " " + merged_data["time"]
merged_data["ISO_formatted_datetime"] = pd.to_datetime(
    datetime_string,
    format="%a %b %d %Y %I:%M:%S %p"
)


# #Columns for lorWan Sensors
temperature_col = [
    col for col in merged_data.columns 
    if "lorawan_readings" in col.lower() and "temperature" in col.lower()
]

humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower() and "lorawan_readings" in col.lower()
]

co2_col = [
    col for col in merged_data.columns 
    if "co2" in col.lower() and "lorawan_readings" in col.lower()
]

sensors_to_keep = ["Sensor_1", "Sensor_3", "Sensor_6"]
sensors_col = [col for col in merged_data.columns if any(sensor in col for sensor in sensors_to_keep)]

weather_cols_to_keep = ["weather_status","weather_temp","weather_humidity"]
weather_col = [col for col in merged_data.columns if any(weathercol in col for weathercol in weather_cols_to_keep)]

#adding avg temp humid and co2
merged_data['avg_temperature'] = merged_data[temperature_col].median(axis=1)
merged_data['avg_humidity'] = merged_data[humidity_col].mean(axis=1)
merged_data['avg_co2'] = merged_data[co2_col].mean(axis=1)

avg_col = [
    col for col in merged_data.columns
    if "avg" in col.lower()
]

#Energy(power,energy,current) Data
energy_data = merged_data[["ISO_formatted_datetime"]+ sensors_col]
energy_data.columns = energy_data.columns.str.replace(
    r"Energy_Readings.Sensor_1\.(Current|Energy|Power)", "compressor_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_3\.(Current|Energy|Power)", "fancoil_1_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_6\.(Current|Energy|Power)", "fancoil_2_\\1", regex=True
)

#indoor Data
indoor_data = merged_data[["ISO_formatted_datetime"] + temperature_col + humidity_col + co2_col + avg_col]


#Weather data
weather_data = merged_data[["ISO_formatted_datetime"]+ weather_col]
weather_data.columns = weather_data.columns.str.replace(
    r"result.weather_status", "weather_status", regex=True
).str.replace(
    r"result.weather_temp", "weather_temp", regex=True
).str.replace(
    r"result.weather_humidity", "weather_humidity", regex=True
)


#merging all the needed data
energy_indoor_merged = pd.merge(energy_data, indoor_data, on='ISO_formatted_datetime', how='inner')
final_merged_data = pd.merge(energy_indoor_merged, weather_data, on='ISO_formatted_datetime', how='inner')

final_merged_data['total_energy'] = (
    final_merged_data['compressor_Energy'] +
    final_merged_data['fancoil_1_Energy'] +
    final_merged_data['fancoil_2_Energy']
)

final_merged_data['total_power'] = (
    final_merged_data['compressor_Power'] +
    final_merged_data['fancoil_1_Power'] +
    final_merged_data['fancoil_2_Power']
)

final_merged_data['total_current'] = (
    final_merged_data['compressor_Current'] +
    final_merged_data['fancoil_1_Current'] +
    final_merged_data['fancoil_2_Current']
)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)




C:\Users\Yun Yang\AppData\Local\Temp\ipykernel_16188\1111285534.py:20: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


In [8]:
print(final_merged_data[["ISO_formatted_datetime",'avg_temperature','total_power','total_current']])

     ISO_formatted_datetime  avg_temperature  total_power  total_current
0       2024-11-15 16:25:07        23.500000     6.647800      10.704001
1       2024-11-15 16:30:06        23.500000     0.218600       0.938000
2       2024-11-15 16:35:06        23.650000     0.105500       0.770000
3       2024-11-15 16:40:05        23.775000     0.106200       0.772000
4       2024-11-15 16:45:06        23.900000     0.106600       0.771000
5       2024-11-15 16:50:06        24.100000     0.099400       0.770000
6       2024-11-15 16:55:06        24.300000     0.099800       0.771000
7       2024-11-15 17:00:05        24.400000     0.099500       0.789000
8       2024-11-15 17:05:06        24.500000     0.099700       0.780000
9       2024-11-15 17:10:06        24.650000     0.099100       0.765000
10      2024-11-15 17:15:06        24.800000     0.098800       0.765000
11      2024-11-15 17:20:06        24.800000     0.098800       0.761000
12      2024-11-15 17:25:06        24.925000     0.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import joblib

class ACTemperatureOptimizer:
    def __init__(self):
        self.model = None
        self.preprocessor = None
        self.features = ['hour', 'day_of_week', 'avg_temperature', 'avg_humidity', 
                         'avg_co2', 'weather_temp', 'weather_humidity','total_power', 
                         'total_current']

    def prepare_features(self, df):
        df['hour'] = pd.to_datetime(df['ISO_formatted_datetime']).dt.hour
        df['day_of_week'] = pd.to_datetime(df['ISO_formatted_datetime']).dt.dayofweek

        return df

    def create_target(self, df):
        comfort_condition = (
            (df['avg_co2'] < 1000) &  # CO2 level
            (40 <= df['avg_humidity']) & (df['avg_humidity'] <= 60) &  # Humidity
            (df['total_power'] < 10) &  # Power consumption threshold
            (df['total_current'] < 13)   # Current threshold
        )
        
        # Calculate comfortable temperature based on comfort conditions
        comfortable_temp = df.loc[comfort_condition, 'avg_temperature'].mean()
        
        # Dynamic weight calculation based on power and current
        power_weight = 1 - (df['total_power'] / df['total_power'].max())
        current_weight = 1 - (df['total_current'] / df['total_current'].max())
        
        '''
        Target temperature calculation:
        1. If comfort conditions are met, use the actual temperature
        2. Otherwise, calculate a weighted temperature considering:
           - 70% of the comfortable mean temperature
           - 30% of the actual temperature
           - Adjusted by power and current consumption weights
        '''
        target = np.where(
            comfort_condition, 
            df['avg_temperature'], 
            0.7 * comfortable_temp + 0.3 * df['avg_temperature'] * (power_weight + current_weight) / 2
        )
        
        return target

    def train(self, df):
        df = self.prepare_features(df)
        X = df[self.features]

        y = self.create_target(df)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.preprocessor = ColumnTransformer([('scale', StandardScaler(), self.features)])
        self.model = Pipeline([
            ('preprocessor', self.preprocessor),
            ('regressor', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
        ])

        self.model.fit(X_train, y_train)
        return self.model.score(X_test, y_test)

    def predict(self, current_conditions):
        # Convert input to DataFrame
        if isinstance(current_conditions, dict):
            current_conditions = pd.DataFrame([current_conditions])
    
        # prep input data
        prepared_data = self.prepare_features(current_conditions)
        
        # predict optimal temp
        return self.model.predict(prepared_data[self.features])
        
    def get_recommendations(self, current_conditions):
        # convert input data to dataframe
        if isinstance(current_conditions, dict):
            current_conditions = pd.DataFrame([current_conditions])
        
        # Extract current and other conditions
        current = current_conditions['total_current'].iloc[0]
        optimal_temp = self.predict(current_conditions)[0]
        print(f'The predicted optimal temperature: {optimal_temp}°C')
        
        current_temp = current_conditions['avg_temperature'].iloc[0]
        print(f'The current temperature provided: {current_temp}°C')
        temp_diff = current_temp - optimal_temp

        if temp_diff > 5:
            return {'action': 'rapid_cool', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}
        
        elif 3 < temp_diff <= 5:
            return {'action': 'cool', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}
        
        elif 1 < temp_diff <= 3:
            return {'action': 'slight_cool', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}
        
        elif -1 <= temp_diff <= 1:
            return {'action': 'maintain', 'current_temp': current_temp, 'optimal_temp': optimal_temp}
        
        elif -3 <= temp_diff < -1:
            return {'action': 'slight_increase', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}
        
        elif -5 <= temp_diff < -3:
            return {'action': 'increase', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}
        
        else:  # temp_diff < -5
            return {'action': 'rapid_increase', 'current_temp': current_temp, 'optimal_temp_to_aim_for': optimal_temp}


        

        # Hard conditons
        
        # Check if it's a weekend
        day_of_week = pd.to_datetime(current_conditions['ISO_formatted_datetime']).dt.dayofweek.iloc[0]
        if day_of_week in [5, 6]:  # Saturday or Sunday
            return {'action': 'off', 'temperature': optimal_temp, 'power_state': 'off'}
        
       

optimizer = ACTemperatureOptimizer()
print("Model accuracy:", optimizer.train(final_merged_data))

# Save and load the model
joblib.dump(optimizer, 'models/ac_optimizer_rf.joblib')


Model accuracy: 0.9950314885475092


['models/ac_optimizer_rf.joblib']

In [10]:

# Example conditions
current_conditions = {
    'ISO_formatted_datetime': "2024-11-28 23:20:06",
    'avg_temperature': 28,
    'avg_humidity': 55,
    'avg_co2': 400,
    'weather_temp': 25,
    'weather_humidity': 65,
    'total_current': 0.9,
    'total_power':6
}
loaded_optimizer = joblib.load('models/ac_optimizer_rf.joblib')

for temp in range(16, 31):  
    
    current_conditions['avg_temperature'] = temp
    

    # Get recommendations for the updated temperature
    recommendations = loaded_optimizer.get_recommendations(current_conditions)
    
    # Print the results
    print(f"Temperature: {temp}°C -> AC Recommendations: {recommendations}\n")

The predicted optimal temperature: 22.058423471592192°C
The current temperature provided: 16°C
Temperature: 16°C -> AC Recommendations: {'action': 'rapid_increase', 'current_temp': 16, 'optimal_temp_to_aim_for': 22.058423471592192}

The predicted optimal temperature: 22.058423471592192°C
The current temperature provided: 17°C
Temperature: 17°C -> AC Recommendations: {'action': 'rapid_increase', 'current_temp': 17, 'optimal_temp_to_aim_for': 22.058423471592192}

The predicted optimal temperature: 22.058423471592192°C
The current temperature provided: 18°C
Temperature: 18°C -> AC Recommendations: {'action': 'increase', 'current_temp': 18, 'optimal_temp_to_aim_for': 22.058423471592192}

The predicted optimal temperature: 22.058423471592192°C
The current temperature provided: 19°C
Temperature: 19°C -> AC Recommendations: {'action': 'increase', 'current_temp': 19, 'optimal_temp_to_aim_for': 22.058423471592192}

The predicted optimal temperature: 22.058423471592192°C
The current temperature 

### Fetching data

In [17]:
import requests
import json

headers = {
    'x-api-key': '123'
}

url = 'http://localhost:8080//AI/getAIData'
# Make the GET request with the headers
response = requests.get(url, headers=headers)

data = json.loads(response.content)
result = data['result'][0]

# Convert date and time to ISO format
datetime_string = result['date'] + " " + result['time']

iso_datetime = pd.to_datetime(
    datetime_string,
    format="%a %b %d %Y %I:%M:%S %p"
)

# Calculate average temperature and humidity
lorawan_readings = result['Lorawan_Readings'].values()
avg_temperature = sum(r.get('temperature', 0) for r in lorawan_readings) / len(lorawan_readings)
avg_humidity = sum(r.get('humidity', 0) for r in lorawan_readings) / len(lorawan_readings)

# Calculate average CO2
co2_values = [r.get('co2', 0) for r in lorawan_readings if 'co2' in r]
avg_co2 = sum(co2_values) / len(co2_values) if co2_values else 0

selected_sensors = ['Sensor_1', 'Sensor_3', 'Sensor_6']
energy_readings = [result['Energy_Readings'][sensor] for sensor in selected_sensors if sensor in result['Energy_Readings']]

# Calculate total current and total power
total_current = sum(r['Current'] for r in energy_readings)
total_power = sum(r['Power'] for r in energy_readings)



weather = data['weather'][0]['result']

weather_humidity = weather['weather_humidity']
weather_temp = weather['weather_temp']


# Construct the desired dictionary
current_conditions = {
    'ISO_formatted_datetime': iso_datetime,
    'avg_temperature': round(avg_temperature, 2),
    'avg_humidity': round(avg_humidity, 2),
    'avg_co2': round(avg_co2, 2),
    'weather_temp': weather_temp,
    'weather_humidity': weather_humidity,
    'total_current': round(total_current, 2),
    'total_power': round(total_power, 2)
}

# Print the result
print(current_conditions)
recommendations = loaded_optimizer.get_recommendations(current_conditions)
print("AC Recommendations:", recommendations)


ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /AI/getAIData (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020CE10C9460>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))